Proyecto jiji

In [45]:
!pip install kagglehub


[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
import nltk
import numpy as np
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
import regex as re
import os
import pandas as pd
stemmer = SnowballStemmer('english')
import os
import pickle
import sys
import math
import dbm
import time
import heapq
import json

[nltk_data] Downloading package punkt to C:\Users\Ce
[nltk_data]     mar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [47]:
with open("stopwords-en.txt", encoding="latin1") as file:
   stoplist = [line.rstrip().lower() for line in file]
stoplist += ['?', '-', '.', ':', ',', '!', ';']

def preprocesamiento(texto, stemming=True):
  words = []
  texto = str(texto)
  texto = texto.lower()
  texto = re.sub(r'[^a-zA-Z0-9_À-ÿ]', ' ', texto)
  # tokenizar
  words = nltk.word_tokenize(texto, language='english')
  # filtrar stopwords
  words = [word for word in words if word not in stoplist]
  # reducir palabras (stemming)
  if stemming:
      words = [stemmer.stem(word) for word in words]
  return words


In [48]:
# Funciones AUXILIARES para recuperar la información

def getNumberWithAtributo(dataset_head, atributo):
    if atributo in dataset_head.columns:
        return dataset_head.columns.get_loc(atributo)  # Obtiene la posición del atributo
    return -1  # Retorna -1 si no existe
 

def getResultados(result, path, disk_limit):
    res = []
    for chunk in pd.read_csv(path, chunksize=disk_limit):
        for doc, score in result:
            if doc in chunk.index:
                res.append((chunk.iloc[doc], score))
    return pd.DataFrame(res, columns=['doc_id', 'score'])  
def getResultadosDF(result, path, disk_limit):
    res = []
    for chunk in pd.read_csv(path, chunksize=disk_limit):
        for doc, score in result:
            row = chunk.iloc[doc].copy()
            row['score'] = score  
            res.append(row)
    return pd.DataFrame(res) 
def get_dfTex_Cols(path, row, columnas): #We don´t use it is only ofr simple an very fast testing
    dataset =  pd.read_csv(path)
    res = [dataset.iloc[row, i] for i in columnas]
    return ' '.join(res)  
#Funciones auxiliares para el mergeo:


In [52]:


class SPIMI:
    def __init__(self, index_dataset_path="path",bloques_dir="index_blocks" , columnas = ["track_name","track_artist","lyrics", "track_album_name"]):
        self.bloques_dir = bloques_dir  
        self.dataset_path = index_dataset_path  
        self.block_counter = 0      
        self.doc_ids = None  
        self.idf = {}
        self.length = {}
        self.disk_limit = 40000  
        
        #--Columnas:
        self.columnas = columnas 
        # ----Direciones de memoria para los files--
        self.dirIndex = self.bloques_dir + "_merge"
        
        self.term_dict_path = os.path.join(self.dirIndex, 'term_dict.pkl')
        self.postings_file_path = os.path.join(self.dirIndex, 'postings.bin')
        self.norms_file_path = os.path.join(self.dirIndex, 'document_norms.pkl')
        #Guardar los ids
        self.doc_count_path = os.path.join(self.dirIndex, 'doc_count.txt')  

        if not os.path.exists(self.dirIndex):
            os.makedirs(self.dirIndex)
        if not os.path.exists(self.bloques_dir):
            os.makedirs(self.bloques_dir)

        if os.path.exists(self.doc_count_path): # Guardamos el total de docs en ves de usar un set
            with open(self.doc_count_path, 'r') as f:
                self.doc_ids = int(f.read().strip())

        self.load_doc_ids()   

    def load_doc_ids(self):
        if os.path.exists(self.doc_count_path):
            try:
                with open(self.doc_count_path, 'r') as f:
                    self.doc_ids = int(f.read().strip())
                    print(f"Documentos cargados: {self.doc_ids}")
            except (ValueError, FileNotFoundError):
                self.doc_ids = 0  # Si el archivo está vacío o corrupto
        else:
            self.doc_ids = 0
 
       
        
                  

    def spimi_invert(self):
        print("Creando y ordenando cada bloque")
        dictionary = {}
        doc_ids = set()
        time_spimiInvert_start = time.time()

        #Get columna indeces: 
        with open(self.dataset_path, mode='r', encoding='utf-8') as file:
            primera_linea = file.readline().strip()
        #Get number columns
        columnas = primera_linea.split(',')
        columnas_numbers  = [i for i in range(len(columnas)) if columnas[i] in self.columnas]

        # Creamos los diccionarios a los bloques
        for chunk in pd.read_csv(self.dataset_path,chunksize= self.disk_limit): 
            for doc_id_, row in chunk.iterrows():
                self.doc_ids +=1 # Actualizamos la cantidad de id
                preFila = [str(row.iloc[i]) for i in columnas_numbers]
                texto = ' '.join(item for item in preFila)
                words = preprocesamiento(texto)
                for text in words:
                    doc_id = doc_id_
                    token = text
                    if token not in dictionary:
                        dictionary[token] = {}  

                    if doc_id not in dictionary[token]:
                        dictionary[token][doc_id] = 1  
                    else:
                        dictionary[token][doc_id] += 1  

                    dictionary_size = sys.getsizeof(dictionary)
                    if dictionary_size >= self.disk_limit:
                        self.write_block_to_disk(dictionary)
                        dictionary.clear()

        if dictionary:
            self.write_block_to_disk(dictionary)  
        
        with open(self.doc_count_path, 'w') as f:
            f.write(str(self.doc_ids))

        time_spimiInvert = time.time()- time_spimiInvert_start

        # MERGE
        print("Iniciando con el merge")
        time_Merge_start = time.time()
        self.mergeHeap()
        time_Merge = time.time() - time_Merge_start


        print("\nResumen de tiempos:")
        print(f"Tiempo para la creación Creación  y el ordenamiento de los bloques: {time_spimiInvert:.2f} segundos")
        print(f"Tiempo para la ejecución del merge: {time_Merge:.2f} segundos")
        print(f"Tiempo total para la creación y el mergeo: {time_Merge + time_spimiInvert:.2f} segundos")
    


    def write_block_to_disk(self, dictionary): # Guardo los ordenados
        # TO DO
        # Probar con guardar y luego ordenar con hilos
        sorted_terms = dict(sorted(dictionary.items())) 
        file_path = os.path.join(self.bloques_dir, f"block_{self.block_counter}.txt")
        with open(file_path, "w", encoding="utf-8") as f:
            for term, postings in sorted_terms.items():
                postings_str = json.dumps(postings)  
                f.write(f"{term}: {postings_str}\n")  
        self.block_counter += 1


    def load_term_dict(self):
        with open(self.term_dict_path, 'rb') as f:
            while True:
                try: #Yiel es un generador, en vez de un return yield mantienes q en piclek esta 
                    yield pickle.load(f) #Avanzamos bloque por bloque, es decir dicionario por diccionario
                except EOFError:
                    break


    def show_Block(self, index):
        print("Imprimiendo contenido del bloque: ", index)
        file_path = os.path.join(self.bloques_dir, f'block_{index}.pkl')
        if not os.path.exists(file_path):
            print(f"El bloque {index} no existe en la ruta: {file_path}")
            return
        try:
            with open(file_path, 'rb') as f:
                block_content = pickle.load(f)
            print(block_content)
        except Exception as e:
            print(f"Error al abrir el bloque {index}: {e}")



    # Ya se comprobó el correcto merge 
    # Ahora los archivo son terminos postings y salto de linea, asi se getea linea por línea 
    def mergeHeap(self):
        lista_bloques = os.listdir(self.bloques_dir)
        priority_queue = []
        idx = 0 
    
        postings_file_path = os.path.join(self.dirIndex, 'postings.bin')
        term_dict_path = os.path.join(self.dirIndex, 'term_dict.pkl')
        norms_file_path = os.path.join(self.dirIndex, 'document_norms.pkl')
    
        open_files = {}
        doc_norms_temp = {}  
    
        #Para optimizar en término s de manejo de memoria secundaria a term_dict.pkl haremos lo siguiente: 
        # en un diccionario temporal y si se supera a disk_limit se guarda
        # las el mañao del diccionario y le diccionario, de tal forma que luego se puda abrir por bloques
        # En términos prácticos es (pos, tamanio) diccionario tempora ...
        temp_term_dict = {}
        temp_term_dict_position = 0

        try:
            for bloque_path in lista_bloques:  # Referencias de bloques y pusheo de una línea (term, postings)
                bloque_full_path = os.path.join(self.bloques_dir, bloque_path)
                open_files[bloque_path] = open(bloque_full_path, "r", encoding="utf-8")
                term_postings = self.read_next_term(open_files[bloque_path])
                if term_postings:
                    term, postings = term_postings
                    heapq.heappush(priority_queue, (term, idx, postings, bloque_path))
                    idx += 1
    
            current_term = None
            current_postings = {}
            postings_file_position = 0
    
            with open(postings_file_path, 'wb') as postings_file, \
                 open(term_dict_path, 'ab') as term_dict_file:
    
                while priority_queue:
                    term, _, postings, bloque_path = heapq.heappop(priority_queue)
                    file = open_files[bloque_path]
    
                    if term == current_term:  # Caso en que el término es igual
                        for doc_id, tf in postings.items():
                            current_postings[doc_id] = current_postings.get(doc_id, 0) + tf
                    else:  # Nuevo término
                        if current_term is not None:
                            # Calcular IDF
                            idf = math.log10(self.doc_ids / (1 + len(current_postings)))
    
                            # Guardar postings (TF e IDF) en postings.bin
                            tf_idf_postings = {}
                            for doc_id, tf in current_postings.items():
                                tf_weighted = math.log10(1 + tf)
                                tfidf = tf_weighted * idf
                                tf_idf_postings[doc_id] = {"tf": tf_weighted, "idf": idf}
    
                                # Acumular el cuadrado del TF-IDF en las normas temporales
                                doc_norms_temp[doc_id] = doc_norms_temp.get(doc_id, 0) + tfidf ** 2
    
                            postings_data = pickle.dumps(tf_idf_postings)
                            postings_file.write(postings_data)
    
                            # Guardar término en term_dict_temporal: 
                            temp_term_dict[current_term] =  (postings_file_position, len(postings_data)) 
                            postings_file_position += len(postings_data)
                        
                        # Si se puera el limite ------><------
                        dictionary_size = sys.getsizeof(temp_term_dict)
                        if dictionary_size >= self.disk_limit: 
                            pickle.dump(temp_term_dict, term_dict_file) 
                            
                            temp_term_dict.clear()
                        
                        # Actualizar current_term y current_postings
                        current_term = term
                        current_postings = postings.copy()

                    next_term_postings = self.read_next_term(file)
                    if next_term_postings:
                        next_term, next_postings = next_term_postings
                        heapq.heappush(priority_queue, (next_term, idx, next_postings, bloque_path))
                        idx += 1
    
                
                if current_term is not None:
                    idf = math.log10(self.doc_ids / (1 + len(current_postings)))
                    tf_idf_postings = {}
                    for doc_id, tf in current_postings.items():
                        tf_weighted = math.log10(1 + tf)
                        tfidf = tf_weighted * idf
                        tf_idf_postings[doc_id] = {"tf": tf_weighted, "idf": idf}
    
                        # Acumular el cuadrado del TF-IDF en las normas temporales
                        doc_norms_temp[doc_id] = doc_norms_temp.get(doc_id, 0) + tfidf ** 2

                    

                    postings_data = pickle.dumps(tf_idf_postings)
                    postings_file.write(postings_data)
                    temp_term_dict[current_term] = (postings_file_position, len(postings_data))
                    # pickle.dump({current_term: (postings_file_position, len(postings_data))}, term_dict_file)
                    postings_file_position += len(postings_data)

                    #Guardamos de frente temp_term_dict
                    pickle.dump(temp_term_dict, term_dict_file) #Guardo el diccionario
                    temp_term_dict.clear()

                    sorted_terms = list(temp_term_dict.keys())
                    
                    temp_term_dict.clear()
    
        finally:
            for file in open_files.values():
                file.close()
    
        # Guardar las normas finales tomando la raíz cuadrada
        final_norms = {int(doc_id): math.sqrt(value) for doc_id, value in doc_norms_temp.items()}
        with open(norms_file_path, 'wb') as norms_file:
            pickle.dump(final_norms, norms_file)
    
        self.term_dict_path = term_dict_path
        self.postings_file_path = postings_file_path
        print(f"Índice fusionado guardado en {postings_file_path}")
        print(f"Diccionario de términos guardado en {term_dict_path}")
        print(f"Normas de documentos guardadas en {norms_file_path}")
            # Realiza merge de todos los bloques:
            # Manejaremos un archivo para ubicar los un indice general self.indexTerms donde se guarda la posición del termino
            # En el otro archivo se guardaran completito tanto la palabra como el posting list pero ya ordenado
    
    def read_next_term(self, file):
        line = file.readline()
        if not line:  # Si llegamos al final del archivo
            return None
        term, postings_str = line.strip().split(":", 1)
        postings = json.loads(postings_str)  
        return term, postings

    #--Funciones para testing--
    def print_all_norms(self):
        if not os.path.exists(self.norms_file_path):
            print("El archivo de normas no existe. Asegúrate de haber ejecutado 'spimi_invert' correctamente.")
            return

        try:
            with open(self.norms_file_path, 'rb') as norms_file:
                norms = pickle.load(norms_file)
                if not norms:
                    print("El archivo de normas está vacío.")
                    return

                print("Normas de los documentos:")
                for doc_id, norm in sorted(norms.items()):
                    print(f"Documento ID: {doc_id}, Norma: {norm}")
        except Exception as e:
            print(f"Error al leer o procesar las normas: {e}")


    def show_terms_and_positions(self):
        print("Términos y sus posiciones en el archivo de postings:")
        all_terms = []  
        # for term, (position, length) in term_dict.items():
        #     print(f"Término: '{term}', Posición: {position}, Longitud: {length}")

        for term_dict in self.load_term_dict():  
            terms = list(term_dict.keys())
            all_terms.extend(terms)
        if all_terms == sorted(all_terms):
            print("----->Yeeeei, Los términos están ordenados alfabéticamente en el diccionario final.")
        else:
            print("Los términos NO están ordenados alfabéticamente en el diccionario final.")


    def show_terms_with_postings(self):
        if not os.path.exists(self.postings_file_path):
            print("El archivo de postings no existe. Ejecuta 'mergeHeap' primero.")
            return

        print("Términos y sus listas de postings:")
        try:
            with open(self.postings_file_path, 'rb') as postings_file:
                for term_dict in self.load_term_dict(): 
                    for term, (position, length) in term_dict.items():
                        postings_file.seek(position)  
                        postings_data = postings_file.read(length) 
                        postings = pickle.loads(postings_data) 
                        print(f"Término: '{term}', Postings: {postings}")
        except Exception as e:
            print(f"Error al mostrar los términos y postings: {e}")


    def retrieval(self, query, k):
        # Cargamos doc_ids si no está ya cargado
        if self.doc_ids is None:
            self.load_doc_ids()
    
        if self.doc_ids == 0:  
            raise ValueError("El índice no está construido o no contiene documentos. Ejecuta 'spimi_invert' antes de realizar consultas.")
        print("Se cargó en total: ", self.doc_ids, " documentos")
    
        # Preprocesar la consulta
        terms = preprocesamiento(query)
    
        tf_query = {}
        for term in terms:
            if term in tf_query:
                tf_query[term] += 1
            else:
                tf_query[term] = 1
    
        tfidf_query = {}
        norm_query = 0
    
        scores = [0] * self.doc_ids  
    
        with open(self.postings_file_path, 'rb') as postings_file, \
             open(self.norms_file_path, 'rb') as norms_file:
    
            for term_dict_block in self.load_term_dict(): # bloque por bloque gracias a yield
                for term, tf in tf_query.items():
                    if term in term_dict_block:  # Considerar términos presentes en este bloque
                        position, length = term_dict_block[term]
    
                        postings_file.seek(position)
                        postings_data = postings_file.read(length)
                        postings = pickle.loads(postings_data)  # Diccionario {doc_id: {'tf': tf, 'idf': idf}}
                        idf = math.log10(self.doc_ids / (1 + len(postings)))
    
                        tfidf_query[term] = math.log10(1 + tf) * idf
                        norm_query += (tfidf_query[term]) ** 2
    
                        w_tq = tfidf_query[term]
                        for doc_id_str, values in postings.items():
                            doc_id = int(doc_id_str)
                            tfidf_td = values['tf'] * values['idf']
                            scores[doc_id] += tfidf_td * w_tq 
    
            norm_query = math.sqrt(norm_query)
    
            norms_file.seek(0)
            norms = pickle.load(norms_file)
    
            for doc_id in range(self.doc_ids):
                doc_norm = norms.get(doc_id, 0)
                if doc_norm != 0:
                    scores[doc_id] /= (doc_norm * norm_query)
    
        result = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
        return result[:k]
    
    




path = "spotify_2000.csv"
# ---------Creation---------
columnas = ["track_name","track_artist","lyrics", "track_album_name"]
s = SPIMI( path, columnas=columnas)  
s.spimi_invert() 

# ---------End Creation---------
print("Se procesaron en total:  ", s.doc_ids)


#Prubas de merge
s.show_terms_and_positions()
# s.show_terms_with_postings()
    

Creando y ordenando cada bloque
Iniciando con el merge
Índice fusionado guardado en index_blocks_merge\postings.bin
Diccionario de términos guardado en index_blocks_merge\term_dict.pkl
Normas de documentos guardadas en index_blocks_merge\document_norms.pkl

Resumen de tiempos:
Tiempo para la creación Creación  y el ordenamiento de los bloques: 30.63 segundos
Tiempo para la ejecución del merge: 1.52 segundos
Tiempo total para la creación y el mergeo: 32.15 segundos
Se procesaron en total:   2000
Términos y sus posiciones en el archivo de postings:
----->Yeeeei, Los términos están ordenados alfabéticamente en el diccionario final.


In [50]:


# ----------Set Atributo--------
query = "this is my breath"
query = get_dfTex_Cols(path, 2, [1,2,3,6])
top_k = 20

result = s.retrieval(query, top_k)
respuesta = getResultados(result, s.dataset_path, s.disk_limit) # La respuesta es un array pd de [fila del dataframe, score]
#-----------End of query------------

# s.print_all_norms()
getResultadosDF(result, s.dataset_path, s.disk_limit)

# print( "Scores: ")
# print( respuestadf["score"])

Se cargó en total:  2000  documentos


,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language,score
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,0,0.2160,0.004320,0.007230,0.4890,0.650,111.904,262467,en,1.000000
622,0dDifktDnbzmGwliFzI0Ld,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,5AR2bbUUDI2XEMarSbs8Cn,Bell Biv DeVoe Greatest Hits (Remastered),2000,New Jack Swing,3ykXidKLz1eYPvuGoFlD1e,...,0,0.2100,0.002060,0.001830,0.6340,0.783,111.879,261627,en,0.994159
237,06yHcjr58IKJnfEK7ko3sD,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",36,3ExqNzkCrQkkknQbrJFqMd,RnB 1990's,2019-07-19,CSR 103:9 (GTA: SA),4sr2xEhXQR5VuZ0LZX8TQ8,...,0,0.2100,0.001950,0.004170,0.6330,0.775,111.815,261853,en,0.983802
1705,0RnDu3eYJqbFKz6MHv2ajd,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",48,2QC1IsQIUNdEz0zgWanPkN,20th Century Masters: The Millennium Collectio...,2002-01-01,New Jack Swing - 90s R&B fused w Hip Hop,79xd4wnVuKZK4rJMsL2wPa,...,0,0.1920,0.002150,0.002680,0.6270,0.802,111.857,262027,en,0.968680
1199,0kYIsBXBR8bg8JN6xuqIDK,Since You Been Gone,RAINBOW,"I get the same old dreams, same time every nig...",6,4saAsJgmmKqFWTzaTdiYzC,Rock,2003-01-01,70s Pop & Rock Hits and Deep Tracks,1uKFRCQYci8kVgMy3xzTVH,...,1,0.0370,0.362000,0.000004,0.5590,0.934,120.875,194000,en,0.120369
703,0EkB09i9ohIUnsw45jzEyf,Toxic,Yael Naim,"Baby, can't you see I'm calling A guy like you...",4,09Cvd1XS0KwCXfMXukpvSS,Yael Naim,2007-10-22,Bluegrass Covers,37i9dQZF1DX56crgoe4TG3,...,1,0.0632,0.605000,0.000056,0.0685,0.157,120.043,267226,en,0.111347
1418,0nrWZWkJM62ftQJSI8fDc0,Diamond Heart,Alan Walker,"Hello, sweet grief I know you'll be the death ...",69,3nzuGtN3nXARvvecier4K0,Different World,2018-12-14,Dance Pop Tunes,4SdfG4cPG3skmTiQLozZGh,...,1,0.0345,0.123000,0.000000,0.4100,0.287,89.909,240333,en,0.110368
501,0bTjMx79m5NMCsHDTyAwBY,Already Dead,Hollywood Undead,It's the year of the snake Friends are overrat...,59,250u5qE6ny6ZYUUtlVGYre,Already Dead,2019-10-25,Rock Hard,37i9dQZF1DWWJOmJ7nRx0C,...,1,0.4080,0.000439,0.000000,0.1310,0.303,159.978,236734,en,0.109022
1875,0TiC3GtlMCskf2hIUIBcDV,Crew Love,Drake,Take your nose off my keyboard What you bother...,51,63WdJvk8G9hxJn8u5rswNh,Take Care (Deluxe),2011-11-15,Urban Contemporary,4Pbs84EQbuAblxlp6Chz0d,...,0,0.2380,0.227000,0.000000,0.2290,0.294,160.152,208813,en,0.102159
1994,0V4l4GQhgnWQGtCWpvA7va,Crew Love,Drake,Take your nose off my keyboard What you bother...,69,6X1x82kppWZmDzlXXK3y3q,Take Care (Deluxe),2011-11-15,PROJECT: Contemporary,6HaCi9bqaiuSZEDfCEmwyo,...,0,0.2380,0.227000,0.000000,0.2290,0.294,160.152,208813,en,0.102159
